In [1]:
from functools import partial, reduce
import numpy as np
import pandas as pd
import fitpack
import interface
import obslib.sidis.reader
from qcdlib.auxiliary import AUX
from qcdlib.collinearity import Collinearity
from tools.config import conf
%matplotlib notebook 

In [2]:
def get_data():
    # Merge all of the data tables
    return reduce(partial(pd.DataFrame.merge, how="outer"), map(pd.DataFrame, conf["sidis tabs"].values()))

In [3]:
input_file = "../inputs/upol_hermes_no_filters.py"

In [4]:
interface.gen_config(input_file)  # read file

conf["aux"] = AUX()
conf["sidis tabs"] = obslib.sidis.reader.READER().load_data_sets("sidis")

data = get_data()

loading sidis data sets 1007

In [5]:
print(", ".join(sorted(data.columns)))

Ebeam, F2, Q2, R, R0, R1, R2, W2, col, dy, hadron, i, lnR, obs, pT, qT, stat_u, systabs_u, systole, systrel, target, value, x, xn, y, yf, yh, yi, yp, z, zn


In [6]:
ki_n = 10
kf_n = 10
kT_n = 10

k_sets_n = ki_n * kf_n * kT_n

kis = np.linspace(0, 2, ki_n)
kfs = np.linspace(0, 2, kf_n)
kTs = np.linspace(0, 2, kT_n)

def Rs(point, ki, kf, kT):
    co = Collinearity(hadron=point["hadron"], ki=ki, kf=kf, kT=kT)
    R0 = co.get_R0(point["Q2"])
    R1 = co.get_R1(point["x"], point["z"], point["Q2"], point["pT"])  # XXX
    R2 = co.get_R1(point["x"], point["z"], point["Q2"], point["pT"])
    return R0, R1, R2

def R_mags(point):
    for ki in kis:
        for kf in kfs:
            for kT in kTs:
                R0, R1, R2 = Rs(point, ki, kf, kT)
                yield np.sqrt(R0 ** 2 + R1 ** 2 + R2 ** 2)

def proximity(point):
    return sum(mag for mag in R_mags(point) if mag <= 1) / k_sets_n

In [7]:
_, p = next(data.iterrows())

p

i                    0
Ebeam             27.6
x            0.0375884
y             0.641938
z             0.152202
Q2             1.24973
pT            0.110571
obs                  M
value          4.39848
stat_u       0.0575923
systrel              0
systabs_u     0.350294
target          proton
hadron             pi+
col             hermes
F2                   0
W2             32.8795
yh           0.0273122
yp             3.45654
yi             0.80461
yf            -0.80461
dy             3.42923
R              1.59268
lnR           0.465419
R0           0.0720157
R1           0.0117231
R2            0.847018
zn            0.151891
xn            0.151891
qT            0.727962
systole            NaN
Name: 0, dtype: object

In [8]:
print("proximity(p):", proximity(p))

print("min |R|:", min(R_mags(p)))

print("avg |R|:", sum(R_mags(p)) / k_sets_n)

print("max |r|:", max(R_mags(p)))

proximity(p): 0.0
min |R|: 1.065760808899371
avg |R|: 3.087278553113723
max |r|: 7.854623797971801


In [9]:
data["min |R|"] = [min(R_mags(p)) for _, p in data.iterrows()]

data["min |R|"]

0       1.065761
1       2.034025
2       2.238552
3       2.370034
4       2.482039
          ...   
2655    0.037983
2656    0.024791
2657    0.005616
2658    0.006310
2659    0.048425
Name: min |R|, Length: 2660, dtype: float64

In [11]:
sum(data["min |R|"] < 1.0)

1872

In [14]:
data["avg |R|"] = [sum(R_mags(p)) / k_sets_n for _, p in data.iterrows()]

data["avg |R|"]

0       3.087279
1       3.880518
2       3.473117
3       3.430131
4       3.453348
          ...   
2655    0.283937
2656    0.286266
2657    0.298543
2658    0.327153
2659    0.371678
Name: avg |R|, Length: 2660, dtype: float64

In [16]:
data["proximity"] = [proximity(p) for _, p in data.iterrows()]

data["proximity"]

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
2655    0.283937
2656    0.286266
2657    0.298543
2658    0.327153
2659    0.371678
Name: proximity, Length: 2660, dtype: float64

In [17]:
max(data["proximity"])

0.7903947018659896